<a href="https://colab.research.google.com/github/aomiglionni/datascience_python/blob/master/P4DS4D%3B_16%3B_Outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SLIDE 11

In [ ]:
import numpy as np
from scipy.stats.stats import pearsonr
np.random.seed(101)
normal = np.random.normal(loc=0.0, scale= 1.0, size=1000)
print ('Mean: %0.3f Median: %0.3f Variance: %0.3f' % (np.mean(normal),
np.median(normal),
np.var(normal)))

Mean: 0.026 Median: 0.032 Variance: 1.109


SLIDE 12

In [ ]:
outlying = normal.copy()
outlying[0] = 50.0
print ('Mean: %0.3f Median: %0.3f Variance: %0.3f' % (np.mean(outlying),
np.median(outlying), np.var(outlying)))
print ('Pearson''s correlation coefficient: %0.3f p-value: %0.3f' % pearsonr(
normal,outlying))

Mean: 0.074 Median: 0.032 Variance: 3.597
Pearsons correlation coefficient: 0.619 p-value: 0.000


SLIDE 19

In [ ]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
X,y = diabetes.data, diabetes.target

SLIDE 20

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
df = pd.DataFrame(X)
print (df.describe())

           0      1      2      3      4      5      6      7      8      9
count 442.00 442.00 442.00 442.00 442.00 442.00 442.00 442.00 442.00 442.00
mean   -0.00   0.00  -0.00   0.00  -0.00   0.00  -0.00   0.00  -0.00  -0.00
std     0.05   0.05   0.05   0.05   0.05   0.05   0.05   0.05   0.05   0.05
min    -0.11  -0.04  -0.09  -0.11  -0.13  -0.12  -0.10  -0.08  -0.13  -0.14
25%    -0.04  -0.04  -0.03  -0.04  -0.03  -0.03  -0.04  -0.04  -0.03  -0.03
50%     0.01  -0.04  -0.01  -0.01  -0.00  -0.00  -0.01  -0.00  -0.00  -0.00
75%     0.04   0.05   0.03   0.04   0.03   0.03   0.03   0.03   0.03   0.03
max     0.11   0.05   0.17   0.13   0.15   0.20   0.18   0.19   0.13   0.14


SLIDE 24

In [ ]:
from sklearn.preprocessing import StandardScaler
Xs = StandardScaler().fit_transform(X)
o_idx = np.where(np.abs(Xs)>3)
# .any(1) method will avoid duplicating
print (df[(np.abs(Xs)>3).any(1)])

        0     1     2     3     4     5     6     7     8     9
58   0.04 -0.04 -0.06  0.04  0.01 -0.06  0.18 -0.08 -0.00 -0.05
123  0.01  0.05  0.03 -0.00  0.15  0.20 -0.06  0.19  0.02  0.07
216  0.01  0.05  0.04  0.05  0.05  0.07 -0.07  0.15  0.05  0.05
230 -0.04  0.05  0.07 -0.06  0.15  0.16  0.00  0.07  0.05  0.07
256 -0.05 -0.04  0.16 -0.05 -0.03 -0.02 -0.05  0.03  0.03  0.01
260  0.04 -0.04 -0.01 -0.06  0.01 -0.03  0.15 -0.08 -0.08 -0.02
261  0.05 -0.04 -0.04  0.10  0.04 -0.03  0.18 -0.08 -0.01  0.02
269  0.01 -0.04 -0.03 -0.03  0.04 -0.01  0.16 -0.08 -0.01 -0.04
322  0.02  0.05  0.06  0.06  0.02 -0.04 -0.09  0.16  0.13  0.08
336 -0.02 -0.04  0.09 -0.04  0.09  0.09 -0.06  0.15  0.08  0.05
367 -0.01  0.05  0.17  0.01  0.03  0.03 -0.02  0.03  0.03  0.03
441 -0.05 -0.04 -0.07 -0.08  0.08  0.03  0.17 -0.04 -0.00  0.00


SLIDE 28

In [ ]:
from scipy.stats.mstats import winsorize
Xs_w = winsorize(Xs, limits=(0.05, 0.95))

SLIDE 35

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from pandas.tools.plotting import scatter_matrix
dim_reduction = PCA()
Xc = dim_reduction.fit_transform(scale(X))
print ('variance explained by the first 2 components: %0.1f%' % (sum(dim_reduction.explained_variance_ratio_[:2]*100)))
print ('variance explained by the last 2 components: %0.1f%' % (sum(dim_reduction.explained_variance_ratio_[-2:]*100)))
df = pd.DataFrame(Xc, columns= ['comp_'+str(j+1) for j in range(10)])
first_two = df.plot(kind='scatter', x='comp_1', y='comp_2', c='DarkGray', s=50)
last_two = df.plot(kind='scatter', x='comp_9', y='comp_10', c='DarkGray', s=50)

SLIDE 40

In [ ]:
from sklearn.cluster import DBSCAN
DB = DBSCAN(eps=2.5, min_samples=25, random_state=101)
DB.fit(Xc)

from collections import Counter
print (Counter(DB.labels_),'\n')
print (df[DB.labels_==-1])
Counter({0: 414, -1: 28})

SLIDE 44

In [ ]:
from sklearn import svm
outliers_fraction = 0.01 #
nu_estimate = 0.95 * outliers_fraction + 0.05
auto_detection = svm.OneClassSVM(kernel="rbf", gamma=0.01, degree=3, nu=nu_estimate) auto_detection.fit(Xc)
evaluation = auto_detection.predict(Xc)
print (df[evaluation==-1])